# Twitch Recommendation System

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm

In [3]:
tf.__version__

'2.11.0'

## Data

### Description

This is a dataset of users consuming streaming content on Twitch. We retrieved all streamers, and all users connected in their respective chats, every 10 minutes during 43 days.

### Metadata

Start and stop times are provided as integers and represent periods of 10 minutes. Stream ID could be used to retrieve a single broadcast segment from a streamer (not used in our work).
1. User ID (anonymized)
2. Stream ID
3. Streamer username
4. Time start
5. Time stop

In [6]:
cols = ["user","stream","streamer","start","stop"]
data = pd.read_csv('data/100k_a.csv', names=cols)

In [7]:
data

,user,stream,streamer,start,stop
0,1,33842865744,mithrain,154,156
1,1,33846768288,alptv,166,169
2,1,33886469056,mithrain,587,588
3,1,33887624992,wtcn,589,591
4,1,33890145056,jrokezftw,591,594
...,...,...,...,...,...
3051728,100000,34167930576,mckytv,3582,3583
3051729,100000,34168288656,natehill,3582,3583
3051730,100000,34169594512,fortnite,3582,3584
3051731,100000,34180223072,highdistortion,3709,3711


In [8]:
max_step = max(data.start.max(),data.stop.max())
data.user, _ = pd.factorize(data.user) # Convert names to categorical labels
data.streamer, _ = pd.factorize(data.streamer) # Convert names to categorical labels

num_users = data.user.nunique()
num_streamers = data.streamer.nunique()

print('Num time periods:', max_step)
print("Num users:", num_users)
print("Num streamers:", num_streamers)
print("Num interactions:", len(data))
print("Estimated watch time (hrs):", (data['stop']-data['start']).sum() * 10 / 60)

Num time periods: 6148
Num users: 100000
Num streamers: 162625
Num interactions: 3051733
Estimated watch time (hrs): 1598118.5


In [9]:
def get_data_availability(data, max_step, num_users, num_streamers):
    av_masks = {}
    for s in tqdm.tqdm(range(max_step+1)):
        av_streamers = data[(data.start<=s) & (data.stop>s)].streamer.unique().tolist()
        # Convert to vector
        av_vector = tf.keras.utils.to_categorical(av_streamers, num_classes=num_streamers).max(axis=0)
        av_masks[s] = np.tile(av_vector, num_users).reshape((num_users, num_streamers))

    return av_masks

    # # Compute availability matrix of size (num_timesteps x max_available)
    # max_av   = max([len(v) for _,v in ts.items()])
    # av_tens = torch.zeros(max_step,max_av).type(torch.long)
    # for k,v in ts.items():
    #     av_tens[k,:len(v)] = torch.LongTensor(v)
    # args.av_tens = av_tens.to(args.device)
    # return data_fu

In [10]:
available_streamers = get_data_availability(data, max_step, num_users, num_streamers)

  0%|          | 1/6149 [00:56<95:50:35, 56.12s/it]

In [ ]:
# class ModelFactory(object):
#     def __init__(self, num_users, num_items, latent_dim):
#         self.num_users, self.num_items, self.latent_dim = num_users, num_items, latent_dim
#         self.max_seq_len = 10

#         self.user_embedding = tf.keras.layers.Embedding(num_users, latent_dim, name='user_embedding')
#         self.item_embedding = tf.keras.layers.Embedding(num_items, latent_dim, name='item_embedding')

#         self.lstm1 = tf.keras.layers.LSTM(latent_dim, return_sequences=True)
#         self.lstm2 = tf.keras.layers.LSTM(latent_dim, return_sequences=True)
#         self.lstm3 = tf.keras.layers.LSTM(latent_dim)

#     def embedding_model(self):
#         inputs = tf.keras.layers.Input(shape=(2,))
#         user_ids, item_ids = inputs[:, 0], inputs[:, 1]
#         user_embed = self.user_embedding(user_ids)
#         item_embed = self.item_embedding(item_ids)
#         outputs = tf.keras.layers.Dot(axes=1)(user_embed, item_embed)

#         return tf.keras.Model(inputs, outputs)

#     def recommendation_model(self):
#         inputs = tf.keras.layers.Input(shape=(self.max_seq_len, self.latent_dim))
#         x = self.lstm1(inputs)
#         x = self.lstm2(x)
#         x = self.lstm3(x)
#         return tf.keras.Model(inputs, x)
        
#     def call(self, inputs):
#         user_ids, item_ids = inputs[:, 0], inputs[:, 1]
#         user_embed = self.user_embedding(user_ids)
#         item_embed = self.item_embedding(item_ids)
#         dot_product = tf.reduce_sum(tf.multiply(user_embed, item_embed), axis=1)
#         return dot_product


# # Training code

## Problem

We want to accurately predict which streamer a user consumes at each 10-minute time period. Formally, we model this problem as follows:

Let $N$ be the total numer of users and $T$ be the total number of time periods. At each time period $t\in\{1,\ldots,T\}$, let $A_t$ be the *available* streamers at time period $t$ and $a_{i,t}\in A_t$ be the streamer whose *available* content we recommend to user $i$. Define the **reward** at time period $t$ to be
$$
r(a_{i,t}, b_{i,t})=\begin{cases}
1, & \text{if }a_{i,t} = b_{i,t} \\
0, & \text{else}
\end{cases}
$$
where $b_{i,t}$ is the streamer whose content user $i$ actually consumes in the next time period, $t+1$. We aim to maximize the total reward across all users:
$$
\max_{\{a_{i,j}:i\in [N],j\in[T]\}} \frac{1}{N}\sum_{i=1}^N \frac{1}{T} \sum_{t=1}^T r(a_{i,t},b_{i,t})
$$

As the manager of the recommendation system, we can exploit co-occurring patterns in the observed behavior across users (up to the current time period $t$) in order to predict future user behavior. We hope this approach improves recommendations for users that interact very sparingly with the system.

In [ ]:
class RecommendationModel(object):
    def __init__(self, num_users, num_items, latent_dim) -> None:
        self.user_embedding = tf.Variable(np.random.normal(0,1, shape=(num_users, latent_dim)))
        self.item_embedding = tf.Variable(np.random.normal(0,1, shape=(num_items, latent_dim)))
        self.trainable_variables = [self.user_embedding, self.item_embedding]

        self.optimizer = tf.keras.optimizers.Adam()
        self.dataset = tf.data.Dataset.from_tensor_slices()
        
    @tf.function
    def loss_fn(self, true_scores, predicted_scores, availability_mask):
        return tf.reduce_sum(tf.multiply(predicted_scores - true_scores, availability_mask)**2)

    @tf.function
    def train_step(self, true_scores, predicted_scores, availability_mask):
        with tf.GradientTape() as tape:
            predicted_scores = tf.matmul(self.user_embedding, self.item_embedding, transpose_b=True)
            loss = self.loss_fn(true_scores, predicted_scores, availability_mask)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss

    def run(self, num_steps):
        for ts, (true_scores, predicted_scores, av_mask) in enumerate(self.dataset):
            # get availability mask

            for _ in range(num_steps):
                loss = self.train_step(true_scores, predicted_scores, av_mask)
            print(f'Timestep {ts+1} - Loss: {loss:.4f}')